# Supercritical Steam Cycle Example
Maintainer: Andrew Lee  
Author: Andrew Lee  

This example uses Jupyter Lab or Jupyter notebook, and demonstrates a supercritical pulverized coal (SCPC) steam cycle model.  See the ```supercritical_steam_cycle.py``` to see more information on how to assemble a power plant model flowsheet. Code comments in that file will guide you through the process.

## Model Description

The example model doesn't represent any particular power plant, but should be a reasonable approximation of a typical plant.  The gross power output is about 620 MW.  The process flow diagram (PFD) can be shown using the code below. The initial PFD contains spaces for model results, to be filled in later.

To get a more detailed look at the model structure, you may find it useful to review ```supercritical_steam_cycle.py``` first. Although there is no detailed boiler model, there are constraints in the model to complete the steam loop through the boiler and calculate boiler heat input to the steam cycle.  The efficiency calculation for the steam cycle doesn't account for heat loss in the boiler, which would be a result of a more detailed boiler model.

In [1]:
# pkg_resources is used here to get the svg information from the
# installed IDAES package

import pkg_resources
from IPython.display import SVG, display

# Get the contents of the PFD (which is an svg file)
init_pfd = pkg_resources.resource_string(
    "idaes.models_extra.power_generation.flowsheets.supercritical_steam_cycle",
    "supercritical_steam_cycle.svg",
)

# Make the svg contents into an SVG object and display it.
display(SVG(init_pfd))

## Initialize the steam cycle flowsheet

This example is part of the ```idaes``` package, which you should have installed.  To run the example, the example flowsheet is imported from the ```idaes``` package. When you write your own model, you can import and run it in whatever way is appropriate for you.  The Pyomo environment is also imported as ```pyo```, providing easy access to Pyomo functions and classes.

The supercritical flowsheet example main function returns a Pyomo concrete mode (m) and a solver object (solver).  The model is also initialized by the ```main()``` function.

In [2]:
import pyomo.environ as pyo
from idaes.models_extra.power_generation.flowsheets.supercritical_steam_cycle import (
    main,
    pfd_result,
)
from idaes.core.util.tables import create_stream_table_dataframe

m, solver = main()

Inside the model, there is a subblock ```fs```.  This is an IDAES flowsheet model, which contains the supercritical steam cycle model. In the flowsheet, the model called ```turb``` is a multistage turbine model.  The turbine model contains an expression for total power, ```power```.  In this case the model is steady-state, but all IDAES models allow for dynamic simulation, and contain time indexes.  Power is indexed by time, and only the "0" time point exists.  By convention, in the IDAES framework, power going into a model is positive, so power produced by the turbine is negative.  

The property package used for this model uses SI (mks) units of measure, so the power is in Watts. Here a function is defined which can be used to report power output in MW.

In [3]:
# Define a function to report gross power output in MW
def gross_power_mw(model):
    # pyo.value(m.fs.turb.power[0]) is the power consumed in Watts
    return -pyo.value(model.fs.turb.power[0]) / 1e6


# Show the gross power
gross_power_mw(m)

## Change the model inputs

The turbine in this example simulates partial arc admission with four arcs, so there are four throttle valves.  For this example, we will close one of the valves to 25% open, and observe the result.

In [4]:
m.fs.turb.throttle_valve[1].valve_opening[:].value = 0.25

Next, we re-solve the model using the solver created by the ```supercritical_steam_cycle.py``` script.

In [ ]:
solver.solve(m, tee=True)

Now we can check the gross power output again.

In [ ]:
gross_power_mw(m)

## Creating a PFD with results and a stream table

A more detailed look at the model results can be obtained by creating a stream table and putting key results on the PFD.  Of course, any unit model or stream result can be obtained from the model.

In [7]:
# Create a Pandas dataframe with stream results
df = create_stream_table_dataframe(streams=m._streams, orient="index")

# Create a new PFD with simulation results
res_pfd = pfd_result(m, df, svg=init_pfd)

In [8]:
# Display PFD with results.
display(SVG(res_pfd))

In [9]:
# Display the stream table.
df